
# Range Box Detection + MetaTrader 5 Login (Python Notebook)

This notebook helps you:
- **Log in to MetaTrader 5** via the official Python package
- **Download OHLC data** for a symbol/timeframe
- **Detect ranging sections** where *high–low* stays within a threshold over a rolling lookback (like your TradingView script)
- **(Optional)** Plot the ranges as shaded boxes

> **Note:** To connect to MT5, you must have the **MetaTrader 5 terminal installed** on your machine and the **`MetaTrader5`** Python package available. If the terminal isn't auto-detected, you'll need to pass its full path to `mt5_login()`.


In [4]:
from idlelib.iomenu import errors

from mt5.pip_value import pip_value_usd

# If needed, install packages (uncomment these lines on your machine)
# !pip install MetaTrader5 pandas matplotlib
from config import Config
from datetime import datetime
instrument_map = {
    "XAUUSD":    ("XAUUSD+", 1.5),
    "BTCUSD":    ("BTCUSD", 50),
    "ETHUSD":    ("ETHUSD", 5),
    "NAS100":    ("NAS100", 5),
    "GBPUSD":    ("GBPUSD+", 0.00030),
    "EURUSD":    ("EURUSD+", 0.00030),
    "USDJPY":    ("USDJPY+", 1)
}

from mt5.mt5_login import mt5_login
import pandas as pd

try:
    import MetaTrader5 as mt5
    HAS_MT5 = True
except Exception as e:
    HAS_MT5 = False
    print("MetaTrader5 package is not available in this environment. "
          "Install it with `pip install MetaTrader5` on your machine to use MT5 features.")


config = Config()

success = mt5_login(
    login=config.MT5_LOGIN,
    password=config.MT5_PASSWORD,
    server='VantageFX-Demo',
    path=r'C:\Program Files\MetaTrader 5\terminal64.exe',
    portable=False)
if not success:
    raise SystemExit("Could not connect to MT5. Check credentials/path and try again.")


Terminal connected: MetaTrader 5 | build: 5200
Account: 510249854 | Server: FTMO-Server | Balance: 49246.03


In [ ]:
from twilio.send_notification import send_sms

## MetaTrader 5 Utilities

## Fetch market data and detect ranges **COMMENTED OUT**

In [ ]:
# from mt5.ranges import merge_consecutive_ranges, find_body_ranges
# from mt5.fetch_rates import mt5_fetch_rates
#
#
# def fetch_market_data(instrument: str, timeframe: int, bars: int) -> pd.DataFrame:
#
#     # --- 2) Fetch data (choose timeframe) ---
#     tframe = mt5.TIMEFRAME_M5  # or mt5.TIMEFRAME_M1
#     lookback = 4  # if your data is M5; use 50 if your data is M1
#     # instrument = 'BTCUSD'  # 'GBPUSD+', 'USDJPY+', 'EURUSD+', 'XAUUSD+', 'NAS100', 'BTCUSD'
#     df = mt5_fetch_rates(instrument, timeframe,bars)  # 5520 bars ~ 46 days for M5, ~ 230 days for M1
#     # --- 3) Choose lookback based on timeframe (example: 5m -> 10 bars; 1m -> 50 bars) ---
#     # Set range_size based on instrument
#     if instrument.startswith('BTCUSD'):
#         range_size = 50
#     elif instrument.startswith('NAS100'):
#         range_size = 5
#     elif instrument.startswith('XAUUSD'):
#         range_size = 1.5
#     elif instrument.startswith('GBPUSD'):
#         range_size = 0.00030
#     elif instrument.startswith('USDJPY'):
#         range_size = 0.003
#     else:
#         range_size = 1  # default for other instruments
#     # --- 4) Detect ranges ---
#     ranges = find_body_ranges(df, lookback=lookback, range_size=range_size)
#     if ranges:
#         ranges_df = pd.DataFrame(ranges)
#         merged_df = merge_consecutive_ranges(ranges_df)
#         print(f"Found {len(merged_df)} ranges for {instrument}")
#         # print(ranges_df[["start_idx", "end_idx", "start_ts", "end_ts", "top", "bottom", "mid", "range_value",
#         #                  "duration_bars"]].to_string())
#         return ranges_df
#     # --- 5) Plot (optional) ---
#     # plot_ranges(df, ranges, title=f"Detected Ranges (lookback={lookback}, size=${range_size})")
#
#
#


## Fetch market data

In [3]:
from mt5.fetch_rates import mt5_fetch_rates


# Fetch data for a specific instrument and timeframe
instrument = 'XAUUSD'
timeframe = mt5.TIMEFRAME_M5  # or mt5.TIMEFRAME_M1
bars = 5520  # ~46 days for M5, ~230 days for M1
rates_data = mt5_fetch_rates(instrument, timeframe, 5520)
if rates_data is not None:
    print("data length " + str(len(rates_data)))
    # print(pd.DataFrame(data)[["open", "high", "low", "close","rsi"]].to_string())
    # print(data.head)
    # print(f"Found {len(merged_df)} ranges for {instrument}")
    print(rates_data.head(3200).to_string())
    print(rates_data.tail().to_string())

else:
    print("No data fetched for the specified range.")

Fetching rates for symbol with updated function: XAUUSD
data length 5235
                    open     high      low    close  tick_volume  spread  real_volume  symbol
time                                                                                         
9-8-2025 01:06   3392.57  3396.32  3390.95  3390.99         1028      23            0  XAUUSD
9-8-2025 01:11   3390.90  3396.00  3389.66  3394.93          927      27            0  XAUUSD
9-8-2025 01:16   3395.01  3397.68  3394.52  3396.38          940      23            0  XAUUSD
9-8-2025 01:21   3396.39  3398.69  3395.97  3397.79          852      21            0  XAUUSD
9-8-2025 01:26   3397.84  3399.91  3397.30  3399.22          895      21            0  XAUUSD
9-8-2025 01:31   3399.19  3401.60  3398.93  3400.14          931      21            0  XAUUSD
9-8-2025 01:36   3400.13  3401.09  3394.28  3396.69         1004      21            0  XAUUSD
9-8-2025 01:41   3396.72  3396.95  3384.07  3389.48         1048      27         

## Calculate Body Ranges and merged Ranges

In [ ]:
from mt5.ranges import find_body_ranges
from mt5.merge_ranges import merge_ranges
import pandas as pd


body_range  = find_body_ranges(rates_data, 4,1.5)
print(body_range.head().to_string())
if body_range is not None and not body_range.empty:
    body_range['date'] = pd.to_datetime(body_range['time'], format='%d-%m-%Y %H:%M', errors='coerce').dt.date
    merged_ranges = merge_ranges(body_range)
    start_date = body_range['date'].min()
    end_date = body_range['date'].max()
    print("Start date:", start_date)
    print("End date:", end_date)
    print(f"range count before merge: {len(body_range)}")
    print(f"range count after merge: {len(merged_ranges)}")
else:
    print("No body ranges found for the specified data.")


In [ ]:
# body_range =body_range.iloc[::-1]  # Reverse the order of the DataFrame
body_range

In [ ]:
from mt5.fetch_rates import mt5_fetch_rates_range

data_in_range = mt5_fetch_rates_range("BTCUSD", mt5.TIMEFRAME_M5, "1/8/2025", "15/8/2025")

In [ ]:
body_range  = find_body_ranges(data_in_range, 4,50)
body_range['date'] = pd.to_datetime(body_range['time'], format='%d-%m-%Y %H:%M').dt.date
merged_ranges = merge_ranges(body_range)
start_date = body_range['date'].min()
end_date = body_range['date'].max()
print("Start date:", start_date)
print("End date:", end_date)
print(f"range count before merge: {len(body_range)}")
print(f"range count after merge: {len(merged_ranges)}")

In [ ]:
# from mt5.lot_calculator import calc_lot_size
# !pip install fastapi uvicorn nest_asyncio
# from fastapi import FastAPI, HTTPException
# from mt5.order import OrderResponseModel, serialize_mt5_result, retcode_name, OrderRequest, map_fill, place_order, \
#     map_time
# import nest_asyncio
# nest_asyncio.apply()
# import json
#
# from fastapi.middleware.cors import CORSMiddleware
# from mt5.ranges import find_body_ranges
# from mt5.merge_ranges import merge_ranges
# from fastapi import FastAPI, Query
# from mt5.fetch_rates import mt5_fetch_rates
# from urllib.parse import unquote
#
# import uvicorn
#
# app = FastAPI()
#
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["*"],  # Allow all origins
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )
#
# @app.get("/fetch_symbol_map")
# def fetch_symbol_map():
#     """
#     Fetch the instrument map with symbols and their corresponding range sizes.
#     """
#     return {
#         "data": instrument_map
#     }
#
# @app.get("/calculate_lots")
# def calculate_lots(
#         symbol: str = Query(...),
#         risk_amount: int = Query(...),
#         sl_pips: int = Query(...),
# ):
#     try:
#         symbol = instrument_map[symbol.upper().strip()][0]
#     except Exception as e:
#         print(f"Error fetching symbol: {e}")
#     lots = calc_lot_size(symbol,risk_amount, sl_pips)
#     print(f"Lot sizes: {lots}")
#
#     return {
#         "symbol": symbol,
#         "lots": lots,
#         "risk_amount": risk_amount,
#         "sl_pips": sl_pips
#     }
#
# @app.get("/fetch_tick_data")
# def fetch_tick_data(
#     symbol: str = Query(...)
# ):
#     try:
#         symbol = instrument_map[symbol.strip()][0]
#     except Exception as e:
#         print(f"Error fetching symbol: {e}")
#
#     print(f"Fetching tick data for symbol: {symbol}")
#     tick_info = mt5.symbol_info(symbol)
#     # print(json.dumps(mt5.symbol_info(symbol)._asdict(), indent=2))
#     pip_value = pip_value_usd(tick_info)
#     print(f"PIP value for {symbol}: {pip_value}")
#     if tick_info is not None:
#         return {
#             "data": {
#                 "symbol": symbol,
#                 "bid": tick_info.bid,
#                 "bidHigh": tick_info.bidhigh,
#                 "bidLow": tick_info.bidlow,
#                 "ask": tick_info.ask,
#                 "askHigh": tick_info.askhigh,
#                 "askLow": tick_info.asklow,
#                 "digits": tick_info.digits,
#                 "spread": tick_info.spread,
#                 "point": tick_info.point,
#                 "pipValue":pip_value,
#                 "tradeTickSize": tick_info.trade_tick_size,
#                 "tradeTickValueProfit": tick_info.trade_tick_value_profit,
#                 "tradeTickValue": tick_info.trade_tick_value,
#                 "volumeStep": tick_info.volume_step,
#                 "volumeMin": tick_info.volume_min,
#                 "volumeMax": tick_info.volume_max
#             }
#         }
#     else:
#         raise HTTPException(status_code=404, detail=f"Tick data for {symbol} not found.")
#
#
# @app.get("/fetch_data")
# def fetch_all_data(
#     tf: int = Query(5),
#     candles: int = Query(5520)
# ):
#     all_results = []
#     for inst, (mapped_inst, range_size) in instrument_map.items():
#         print(f"instrument is {mapped_inst}")
#         rates_data = mt5_fetch_rates(mapped_inst, tf, candles)
#         if rates_data is not None:
#             body_range = find_body_ranges(rates_data, 4, range_size)
#             merged_ranges = merge_ranges(body_range)
#             merged_ranges["instrument"] = inst
#             all_results.append(merged_ranges)
#     if all_results:
#         result_df = pd.concat(all_results, ignore_index=True)
#         return {
#             "data_length": len(result_df),
#             "data": result_df.to_dict(orient="records")
#         }
#     else:
#         return {"error": "No data fetched for any instrument."}
#
# @app.get("/fetch_data")
# def fetch_data(
#     instrument: str = Query(...),
#     timeframe: int = Query(5),
#     bars: int = Query(5520)
# ):
#     symbol = unquote(instrument)
#     print(f"Fetching data for {symbol}")
#     range_size = 1.5  # Default range size for XAUUSD
#     if instrument.startswith('BTCUSD'):
#         range_size = 50
#     if instrument.startswith('ETHUSD'):
#         range_size = 50
#     elif instrument.startswith('NAS100'):
#         range_size = 5
#     elif instrument.startswith('XAUUSD'):
#         instrument = "XAUUSD+"
#         range_size = 1.5
#     elif instrument.startswith('GBPUSD'):
#         instrument = "GBPUSD+"
#         range_size = 0.00030
#     elif instrument == "EURUSD":
#         instrument = "EURUSD+"
#         range_size = 0.00030
#     elif instrument.startswith('USDJPY'):
#         instrument = "USDJPY+"
#     else:
#         raise ValueError("Unsupported instrument. Please use one of the supported instruments.")
#
#     print(f"instrument is {instrument}")
#     rates_data = mt5_fetch_rates(instrument, timeframe, bars)
#     body_range  = find_body_ranges(rates_data, 4,range_size)
#     merged_ranges = merge_ranges(body_range)
#     if rates_data is not None:
#         return {
#             "data_length": len(merged_ranges),
#             "data": merged_ranges.to_dict(orient="records")
#         }
#     else:
#         return {"error": "No data fetched for the specified range."}
#
# @app.post("/orders", response_model=OrderResponseModel)
# def create_order(req: OrderRequest):
#     type_filling = map_fill(req.type_filling)
#     type_time = map_time(req.type_time)
#
#     result = place_order(
#         symbol=req.symbol,
#         side=req.side,
#         volume=req.volume,
#         kind=req.kind,
#         price=req.price,
#         stoplimit=req.stoplimit,
#         deviation=req.deviation,
#         sl_points=req.sl_points,
#         tp_points=req.tp_points,
#         sl_price=req.sl_price,
#         tp_price=req.tp_price,
#         magic=req.magic,
#         comment=req.comment,
#         do_order_check=req.do_order_check,
#         type_filling=type_filling,
#         type_time=type_time,
#         expiration=req.expiration,
#     )
#
#     if not result.ok:
#         payload = OrderResponseModel(
#             ok=result.ok,
#             retcode=result.retcode,
#             retcode_meaning=retcode_name(result.retcode),
#             comment=result.comment,
#             result=serialize_mt5_result(result.result),
#         )
#         # Convert to dict for JSON serialization
#         raise HTTPException(status_code=400, detail=payload.dict())
#
#     payload = OrderResponseModel(
#         ok=result.ok,
#         retcode=result.retcode,
#         retcode_meaning=retcode_name(result.retcode),
#         comment=result.comment,
#         result=serialize_mt5_result(result.result),
#     )
#     # For MT5 rejections (positive retcodes but not DONE), still return 200 with details
#     return payload
#
# uvicorn.run(app, host="0.0.0.0", port=8000)

In [25]:


from_date=datetime(2020,1,1)
to_date=datetime.now()
history_orders=mt5.history_orders_get(from_date, to_date)
if history_orders is None:
    print("No orders with group XAUUSD")
elif len(history_orders)==0:
    print("No orders with group XAUUSD")
else:
    columns=history_orders[0]._asdict().keys()
    print(columns)
    df=pd.DataFrame(list(history_orders),columns=history_orders[0]._asdict().keys())
    df.drop(['ticket','time_setup_msc', 'time_done_msc', 'time_expiration', 'time_expiration', 'type_time', 'type_filling', 'state', 'magic', 'position_by_id','volume_current', 'price_open','sl', 'tp', 'price_current', 'price_stoplimit', 'external_id'], axis=1, inplace=True)
    # df['time'] = pd.to_datetime(df['time'], unit='s')
    print(history_orders[0])
    print(df.head())

dict_keys(['ticket', 'time_setup', 'time_setup_msc', 'time_done', 'time_done_msc', 'time_expiration', 'type', 'type_time', 'type_filling', 'state', 'magic', 'position_id', 'position_by_id', 'reason', 'volume_initial', 'volume_current', 'price_open', 'sl', 'tp', 'price_current', 'price_stoplimit', 'symbol', 'comment', 'external_id'])
TradeOrder(ticket=182514785, time_setup=1756721849, time_setup_msc=1756721849406, time_done=1756721849, time_done_msc=1756721849536, time_expiration=0, type=1, type_time=0, type_filling=1, state=4, magic=48, position_id=182514785, position_by_id=0, reason=3, volume_initial=1.0, volume_current=0.0, price_open=0.0, sl=3476.04, tp=3468.54, price_current=3474.24, price_stoplimit=0.0, symbol='XAUUSD', comment='', external_id='182514785-O')
   time_setup   time_done  type  position_id  reason  volume_initial  symbol  \
0  1756721849  1756721849     1    182514785       3             1.0  XAUUSD   
1  1756722498  1756722498     0    182514785       5             1

In [24]:
df

,time_setup,time_done,type,position_id,reason,volume_initial,symbol,comment
0,1756721849,1756721849,1,182514785,3,1.00,XAUUSD,
1,1756722498,1756722498,0,182514785,5,1.00,XAUUSD,[tp 3471.04]
2,1756737201,1756737205,2,182573827,0,1.00,XAUUSD,
3,1756738169,1756738169,1,182573827,4,1.00,XAUUSD,[sl 3468.50]
4,1756738600,1756739330,2,182577798,0,1.00,XAUUSD,
5,1756743153,1756743153,1,182577798,0,1.00,XAUUSD,
6,1756786360,1756786360,1,182673383,3,1.00,XAUUSD,
7,1756786398,1756786398,0,182673383,4,1.00,XAUUSD,[sl 3486.31]
8,1756786767,1756786767,1,182676318,3,1.00,XAUUSD,
9,1756786780,1756786780,0,182676318,4,1.00,XAUUSD,[sl 3493.54]


In [12]:
# get the list of positions on symbols whose names contain "*USD*"
usd_positions=mt5.positions_get(group="*USD*")
if usd_positions==None:
    print("No positions with group=\"*USD*\", error code={}".format(mt5.last_error()))
elif len(usd_positions)>0:
    print("positions_get(group=\"*USD*\")={}".format(len(usd_positions)))
    # display these positions as a table using pandas.DataFrame
    df=pd.DataFrame(list(usd_positions),columns=usd_positions[0]._asdict().keys())
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.drop(['time_update', 'time_msc', 'time_update_msc', 'external_id'], axis=1, inplace=True)
    print(df)